## Hittite language transliteration Shavrina

На сервере в home/guest1/2017/Shavrina/hettencode.py

Использование: python hettencode.py

In [1]:
import sys
import getopt
import argparse
import re
import collections

In [2]:
def read_html(syllab_file):
    file = open(syllab_file, "r", encoding="utf8").read()
    syllabtext = []
    htmlhead = re.split('<body lang="ru-RU" dir="ltr">', file)[0]
    htmlbody = re.split('</p>', re.split("<div id=", file)[0])
    for i in range (len(htmlbody)):
        htmlbody[i] = re.sub("</span></font></sup>","<separator>",htmlbody[i])
        htmlbody[i] = re.sub("</span></font></sub>","<separator>",htmlbody[i])
        htmlbody[i] = re.sub("</span></font>","<separator>",htmlbody[i])
        lst = re.split("<separator>",htmlbody[i])
        for y in lst:
            if "remaining" not in y or "missing" not in y :
                syllabtext.append(y)
    htmlbottom = "</p>\n</body>\n</html>" #отрезаем комментарии
    for i in range(len(syllabtext)):
        if '<a class=' in syllabtext[i]:
            syllabtext[i] = "".join([re.split("<a class=",syllabtext[i])[0], "'"]) #отрезаем сноски

    return syllabtext,htmlhead,htmlbottom


In [3]:
vowels = ['aeiuAEIU']

def token_lang(token,cleantoken):
    #определяем язык слова 
    
    if "<i>" in token:
        lang = 'akkad'
    elif cleantoken.isupper():
        lang = 'shumer'
    else:
        lang = 'hett' #default
    return (lang)



In [4]:
def script(token):
    #определяем написание
    script_type = 'normal'
    if '<sup>' in token:
        script_type = 'superscript'
    elif '<sub>' in token:
        script_type = 'subscript'     
    return(script_type)

In [5]:
def clean_token(token): #только само хеттское слово, без тегов
    fin = re.sub("'","", token)
    fin = re.sub('</i>',"", fin)
    cleantoken1 = re.split(">",fin)
    cleantoken = cleantoken1[-1]
    return(cleantoken)

In [6]:
def html_feat(token):
    try:
        fontsize = re.split('"', re.split('<font size="', token)[1])[0]
        style = re.split('"', re.split('style="', token)[1])[0]
        spanlang = re.split('"', re.split('<span lang="', token)[1])[0]
        return(fontsize, spanlang, style)
    except:
        fontsize = 2
        spanlang = "it-IT"
        style = "font-size: 9pt"
        return(fontsize, spanlang, style)


In [7]:
def lonely_vowels(i,cleantoken):
    if i==0 and len(cleantoken)>1 and cleantoken[i+1]=="-":
        features="lonely,start"
    elif i==(len(cleantoken)-1) and cleantoken[i-1]=="-":
        features="lonely,end"
    elif cleantoken[i-1]=="-" and cleantoken[i+1]=="-":
        features="lonely,mid"
    else:
        features="notlonely"
    return(features)

In [8]:
def operations(token_dict):
    j=0
    new_dict = {}
    garbage = ['*', '~','!', '?', '[', ']', '˹', '˺', '˼', '˻']
    for i in range(len(token_dict)):
        if token_dict[i][0]=="-":
            if  token_dict[i-1][1]==token_dict[i+1][1]:
                new_dict[j] = token_dict[i]
                j+=1
            else:
                token_dict[i][0]=re.sub('-','',token_dict[i][0])
                new_dict[j] = token_dict[i]
                j+=1

        elif token_dict[i][1]=='hett':
            if token_dict[i][0]=='í':
                token_dict[i][0]=re.sub('í','i',token_dict[i][0])
                new_dict[j] = token_dict[i]
                j+=1
            elif token_dict[i][2]=='subscript':
                pass

        elif token_dict[i][0]=='i':
            if  token_dict[i-1][0] in vowels :
                if token_dict[i+1][0] in vowels:
                    token_dict[i][0]=re.sub('i','y',token_dict[i][0])
                    new_dict[j] = token_dict[i]
                    j+=1

        elif token_dict[i][0] in vowels :
            if token_dict[i+1][0]==token_dict[i][0]:
                if token_dict[i][4].startswith('lonely') or token_dict[i+1][4].startswith('lonely'):
                    new_dict[j] = token_dict[i]
                    j+=1
                else:
                    token_dict[i][0]="" #смотрим только вперед, остается только вторая гласная
                    new_dict[j] = token_dict[i]
                    j+=1
                

        elif token_dict[i][0].isalpha()==False:
            pass
        
        elif token_dict[i][0] in garbage:
            pass

        elif token_dict[i][0] in vowels :
            if token_dict[i+1][0]==token_dict[i][0]:
                if i==0 or i==len((token_dict)-1):
                    token_dict[i][0] = re.sub('a','ā',token_dict[i][0])
                    token_dict[i][0] = re.sub('u','ū',token_dict[i][0])
                    token_dict[i][0] = re.sub('e','ē',token_dict[i][0])
                    token_dict[i][0] = re.sub('i','ī',token_dict[i][0])
                    token_dict[i+1][0] = re.sub(token_dict[i+1][0],'',token_dict[i+1][0])
                    new_dict[j] = token_dict[i]
                    j+=1
                elif token_dict[i][4].startswith('lonely') | token_dict[i+1][4].startswith('lonely'):
                    token_dict[i][0] = re.sub('a','ā',token_dict[i][0])
                    token_dict[i][0] = re.sub('u','ū',token_dict[i][0])
                    token_dict[i][0] = re.sub('e','ē',token_dict[i][0])
                    token_dict[i][0] = re.sub('i','ī',token_dict[i][0])
                    token_dict[i+1][0] = re.sub(token_dict[i+1][0],'',token_dict[i+1][0])
                    new_dict[j] = token_dict[i]
                    j+=1
                elif token_dict[i+2][0]==token_dict[i][0]:
                    token_dict[i][0] = re.sub('a','ā',token_dict[i][0])
                    token_dict[i][0] = re.sub('u','ū',token_dict[i][0])
                    token_dict[i][0] = re.sub('e','ē',token_dict[i][0])
                    token_dict[i][0] = re.sub('i','ī',token_dict[i][0])
                    token_dict[i+1][0] = re.sub(token_dict[i+1][0],'',token_dict[i+1][0])
                    token_dict[i+2][0] = re.sub(token_dict[i+2][0],'',token_dict[i+2][0])
                    new_dict[j] = token_dict[i]
                    j+=1
                else:
                    new_dict[j] = token_dict[i]
                    j+=1

        elif token_dict[i][0]=='h':
            token_dict[i][0]=re.sub('h','ḫ',token_dict[i][0])
            new_dict[j] = token_dict[i]
            j+=1

        elif token_dict[i][0]=='s':
            token_dict[i][0]=re.sub('s','š',token_dict[i][0])
            new_dict[j] = token_dict[i]
            j+=1

        else:
            new_dict[j] = token_dict[i]
            j+=1

        return(new_dict)

In [9]:
def restore_token(token,lang,script_type,html):
    # сначала разные языки букв
    restored_token = token
    if lang =='akkad':
        
        restored_token= '<font size="' + str(html[0]) + ' style=' + html[2] + '"><span lang="' + html[1] + '">' + "<i>" + restored_token + "</i>" + '</span></font>'
    else:
        restored_token = '<font size="' + str(html[0]) + ' style=' + html[2] + '"><span lang="' + html[1] + '">' + restored_token + '</span></font>'
            
    if script_type == '<sup>':
        restored_token="<sup>" + restored_token + "</sup>"
    elif script_type == '<sub>':
        restored_token="<sub>" + restored_token + "</sub>"
    return(restored_token)

In [13]:
def syll_to_short(syllabtext):
    shorttext = []

    #сделаем из каждого токена словарь, ключом которого будет порядковый номер буквы,
    # а значениями - сама буква и ее признаки: язык, одиноч и ее позиция, суперскрипт/субскрипт, html
    for token in syllabtext:

        token_dict = {}
        
        cleantoken = clean_token(token)
        if len(cleantoken)>0:

            for j in range(len(cleantoken)):
                if cleantoken[j] in vowels:
                    features = lonely_vowels(j,cleantoken)
                else:
                    features = "cons" 
                token_dict[j] = [cleantoken[j], script(token), token_lang(token,cleantoken),  html_feat(token), features]

            new_dic = operations(token_dict)
            print(new_dic)
            word = ""
            for i in range(len(new_dic)):
                word += new_dic[i][0]

                lang, script1, html = new_dic[i][2], new_dic[i][1], new_dic[i][3]

            new = restore_token(word, lang, script1, html)
            word = ""
            shorttext.append(new)
    return (shorttext)


In [11]:
def create_html(syllab_file, short_file):
    syllabtext,htmlhead,htmlbottom = read_html(syllab_file)

    htmlhead += '<body lang="ru-RU" dir="ltr">\n'
    shorttext = syll_to_short(syllabtext)
    html = htmlhead
    for i in range(len(shorttext)):
        html += shorttext[i]
    html += htmlbottom
    outfile = open(short_file, "w", encoding="utf8")
    outfile.write(html)

In [14]:
if __name__ == "__main__":
    
    #parser = argparse.ArgumentParser()
    #args = parser.parse_args()
    #if len(args) != 2:
        #sys.exit("Использование: python hettencode.py syllab_file short_file")
    #syllab_file, short_file = args
    syllab_file = r"C:\Users\USER\Documents\hse\progr\syllab1.html"
    short_file = r"C:\Users\USER\Documents\hse\progr\syllab1_short.html"
    create_html(syllab_file, short_file)

    

{}


UnboundLocalError: local variable 'lang' referenced before assignment